In [1]:
import numpy as np 
import os 

In [2]:
def load_file(path):
    with open(path, 'r') as f:
        data = f.readlines()
    data = [[int(i) for i in j.strip().split(' ')] for j in data]
    data_f = []
    for line in data:
        uid = line[0]
        for item in line[1:]:
            data_f.append([uid, item])
    data_f = np.array(data_f)
    n_user = len(list(set(data_f[:,0])))
    n_item = len(list(set(data_f[:,1])))    
    return data, data_f, n_user, n_item

_, train_data, n_user_train, n_item_train = load_file('train.txt')
print(f'num sample train: {len(train_data)}') # ok 
# print(f'train data:\n{train_data}')
print(f'n user train: {n_user_train}\nn item train: {n_item_train}')
print('-'*30)
_, test_data, n_user_test, n_item_test = load_file('test.txt')
print(f'num sample test: {len(test_data)}') 
# print(f'test data:\n{test_data}')
print(f'n user test: {n_user_test}\nn item test: {n_item_test}')
print('-'*30)
_,val_data, n_user_val, n_item_val = load_file('val.txt')
print(f'num sample val: {len(val_data)}')
# print(f'val data:\n{val_data}')
print(f'n user val: {n_user_val}\nn item val: {n_item_val}')

n_item = n_item_train + n_item_test + n_item_val # item in dataset 
print(f'n item: {n_item}')
user_list = np.concatenate([train_data[:,0], test_data[:,0], val_data[:,0]])
print(f'len cat: {len(user_list)}')
user_set = np.unique(user_list)
n_user = len(user_set)
n_warm_user = len(list(set(train_data[:,0])))
n_warm_item = len(list(set(train_data[:,1])))
print(f'n user: {n_user}')
print(f'n warm user: {n_warm_user}')
print(f'n warm item: {n_warm_item}')
print(f'max uid: {np.max(user_set)}')
print(f'min uid: {np.min(user_set)}')

print(f'max iid: {np.max([np.max(train_data[:,1]), np.max(test_data[:,1]), np.max(val_data[:,1])])}')
print(f'min iid: {np.min([np.min(train_data[:,1]), np.min(test_data[:,1]), np.min(val_data[:,1])])}')

# create mapping function
item_map = np.arange(n_item) + n_user
# train_data[:,1] = item_map[train_data[:,1]]
train_data[:,1] += n_user 
# test_data[:,1] = item_map[test_data[:,1]]
print(f'N_USER: {n_user}')
print(f'MAXXXX: {np.min(test_data[:,1]),np.min(test_data[:,0])}')
test_data[:,1] += [n_user]*len(test_data) 
print(f'MAXXXX: {np.min(test_data[:,1]), np.min(test_data[:,0])}')
# val_data[:,1] = item_map[val_data[:,1]]
val_data[:,1] += n_user 


# size data  
size_data = np.array([n_user, n_item, n_warm_user, n_warm_item])
np.save('baby/size_data.npy', size_data)

# save file 
if not os.path.exists('baby'):
    os.mkdir('baby')
np.save(f'baby/train.npy', train_data)
np.save(f'baby/test_cold.npy', test_data)
np.save(f'baby/val_cold.npy', val_data)

# user set have id: [0, n_user-1]
# item set have id: [n_user, n_user+n_item-1]

# create mapping function 
# current: user id: [0, n_user-1] ~ [0, 19444] 
# current: item id: [0, n_item-1] ~ [0, 7049] 
# need: user id: [0, n_user-1] ~ [0, 19444]
# need: item id: [0, n_item-1] ~ [19445, 26493]


# print(np.max(item_map), np.min(item_map))

# check 

# train_data 
# train_data[:,1] = item_map[train_data[:,1]]
# print(train_data[:,1][0])
# print(item_map[train_data[:,1]][0])
# print(3239 + n_user)
# ok 

# apply 

num sample train: 128963
n user train: 19442
n item train: 5640
------------------------------
num sample test: 15982
n user test: 10474
n item test: 705
------------------------------
num sample val: 15847
n user val: 10342
n item val: 705
n item: 7050
len cat: 160792
n user: 19445
n warm user: 19442
n warm item: 5640
max uid: 19444
min uid: 0
max iid: 7049
min iid: 0
N_USER: 19445
MAXXXX: (14, 0)
MAXXXX: (19459, 0)


In [3]:
from collections import defaultdict
train_raw = load_file('train.txt')[0]
train_dict = defaultdict(list)
train_dict_raw = defaultdict(list)
for line in train_raw:
    uid = line[0]
    # print(uid, type(uid))
    a = np.array(line[1:]) + n_user 
    train_dict[uid].extend(a)
    train_dict_raw[uid].extend(line[1:])
train_dict_np = np.array(dict(train_dict))
# save 
np.save(f'baby/user_item_all_dict.npy', train_dict_np)
np.save(f'baby/user_item_train_dict.npy', train_dict_np)

In [4]:
# copy multimedia feature from raw folder 
import shutil

# copy file 
list_file = [
    "image_feat.npy",
    "text_feat.npy",
    "image_feat_missing_test.npy",
    "text_feat_missing_test.npy",
    "image_feat_missing_train.npy",
    "text_feat_missing_train.npy",
    "image_feat_missing_all.npy",
    "text_feat_missing_all.npy",
]
# shutil.copy('raw/multimedia_feature.npy', 'format_data/multimedia_feature.npy')

print('starting....')
for file in list_file:
    shutil.copy(f'./{file}', f'baby/{file}')
print('end!')

starting....
end!


In [5]:
# load to test 
user_item_all_dict = np.load('baby/user_item_all_dict.npy', allow_pickle=True).item()
print(user_item_all_dict[0])
print(dict(train_dict_raw)[0])
print(user_item_all_dict[0][0] - dict(train_dict_raw)[0][0] - n_user)

[21194, 20766, 19955, 24415, 23390]
[1749, 1321, 510, 4970, 3945]
0


In [6]:
# cold set 
cold_index = np.load('./cold_item_index.npy')
cold_index = cold_index + n_user 
np.save('baby/cold_set.npy', cold_index)

# warm set 
warm_index = np.array(list(set(train_data[:,1])))
print(f'warm_index: {warm_index}') 
np.save('./baby/warm_set.npy', warm_index)

warm_index: [19445 19447 19448 ... 26489 26492 26494]


In [7]:
# check all 
def check(file_path):
    data = np.load(file_path, allow_pickle=True)
    all_user = data[:,0]
    all_item = data[:,1]
    test1 = np.max(all_item) - np.max(all_user)
    print(f'test 1: {test1}')
    test2 = np.min(all_item) - np.min(all_user)
    print(f'test 2: {test2}')
    print(f'test 3: {np.min(all_item), np.min(all_user)}')

check('baby/train.npy')
check('baby/test_cold.npy')
# check('baby/val_cold.npy')

"""
num_user = 19445
num_item = 7050
num_warm_item = 5640
"""

test 1: 7050
test 2: 19445
test 3: (19445, 0)
test 1: 7048
test 2: 19459
test 3: (19459, 0)


'\nnum_user = 19445\nnum_item = 7050\nnum_warm_item = 5640\n'